In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from ipywidgets import widgets
from scipy.signal import find_peaks

plt.rcParams['figure.figsize'] = (16, 12)

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (16, 12),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

## Periodicity in COVID-19 case data

Eyeballing the world case data there is a fairly obvious periodicity somewhere around 60 days. This notebook explores that pattern a bit more. If you just want to explore the pattern, try `Run -> Run All Cells`, then jump to the [Autocorrelation section](#Autocorrelation) and start exploring the data for different countries.


In [ ]:
dataCSV = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
#dataCSV = './owid-covid-data.csv.bz2'
icDF = pd.read_csv(
    dataCSV,
    index_col='date',
    parse_dates=['date']
)

cDF = icDF.copy()
cDF.index = icDF.index - icDF.index.min()
cDF.head()

Starting with the world data, look at the `new_cases` vs time since first case for the entire world

In [ ]:
wDF = cDF[cDF['location'] == 'World'].dropna(subset=['new_cases'])

plt.plot(wDF.index.days, wDF.new_cases)
plt.title('Daily New Cases - Worldwide')

There's a fairly easy to spot periodicity about 50-100 days in length. Let's take a fourier transform and see if it stands out (toss the first couple of entries because the data are reported ~daily.

In [ ]:
ps = np.square(np.abs(np.fft.rfft(wDF.new_cases)))
d = np.arange((wDF.index.max() - wDF.index.min()).days / 2)
plt.plot(d[2:], ps[-len(d[2:]):])

So there _is_ something there. Since the world data combines lots of reporting it's possible that this is just the data from one or two countries skewing the world data so let's start looking at individual locations. Try changing the dropdown below, here are some interesting countries to try:

### Show the 60 day spike
  * Brazil (wow!)
  * France
  * Germany (wow!)
  * Italy
  * Sweden (wow!)
  * United States
  * United Kingdom (a bit)
  * ...
  
### Don't show the 60 day spike
  * Canada
  * India
  * ...
  

In [ ]:
def powerSpectrumCases(country):
    localDF = cDF[cDF['location'] == country].dropna(subset=['new_cases'])
    if len(localDF.new_cases > 10):
        ps = np.square(np.abs(np.fft.rfft(localDF.new_cases)))
        d = np.arange((localDF.index.max() - localDF.index.min()).days / 2)
        
        fig, ax = plt.subplots(1)
        ax.plot(d[2:], ps[-len(d[2:]):])
        plt.show()

a=widgets.interactive(
    powerSpectrumCases,
    country=widgets.Dropdown(
        options=sorted(set(cDF.location)),
        value = 'Sweden',
        description='Country:',
        disabled=False
    )
)
display(a)

### Aggregating over countries

We can use `find_peaks` to try to identify peak locations then look ad common values for those peaks.

In [ ]:
peaks = []
for country in set(cDF.location):
    localDF = cDF[cDF['location'] == country].dropna(subset=['new_cases'])
    if len(localDF.new_cases > 10):
        ps = np.square(np.abs(np.fft.rfft(localDF.new_cases)))
        d = np.arange((localDF.index.max() - localDF.index.min()).days / 2)
        peaks.append(find_peaks(ps[2:], threshold=max(ps[2:]) * 0.01)[0])

c = Counter(np.concatenate(peaks))
plt.bar(*zip(*c.most_common()), width=.75, color='g')

It's still there!

### Possible explainations

* Is this just an artifact of huge case counts in a few countries? e.g. There is a **huge** spike at ~60 days in Brazil, but it's there in lots of other countries as well. If it was related to the restrictions shouldn't there me more variation (different countries taking different approaches)
* Artifact of the data collection intervals?
* Seasonal artifact?
* Is is a coding or classification issue with the data?
* Other/something a mistake above?

In [ ]:
fig, ax = plt.subplots(5, figsize=(8,8))

# Pick some countries for side-by-side display
for index, country in enumerate(['World','Sweden', 'Germany', 'Brazil', 'Argentina']):
    localDF = cDF[cDF['location'] == country].dropna(subset=['new_cases'])
    if len(wDF.new_cases > 10):
        ps = np.square(np.abs(np.fft.rfft(localDF.new_cases)))
        d = np.arange((localDF.index.max() - localDF.index.min()).days / 2)
        
        ax[index].plot(d[2:], ps[-len(d[2:]):])
        ax[index].set_title(country, fontsize=16)
        ax[index].yaxis.set_ticklabels([])
ax[index].set_xlabel('Days', fontsize=20)
plt.tight_layout()
plt.show()

## Autocorrelation

Looking for the same thing with autocorrelation. Specifically, we will calculate the daily change in new cases, then we will look for autocorrelation in the rolling average of this difference.

In [ ]:
class AutoCorrDisplay:
    
    def __init__(self, country='Canada', ndays=14):
        self.country = country
        self.ndays = ndays
        self.output_widget = widgets.Output()
        self.container = widgets.VBox()
        self.redraw_whole_plot()
        self.draw_app()
        
    def draw_app(self):
        """
        Run once at startup to provide controls and canvas
        """
        self.ndays_slider = widgets.HBox([
                widgets.Label('Rolling avg. window (days):'),
                widgets.IntSlider(
                    min=1,
                    max=60,
                    step=1,
                    value=self.ndays,
                )
        ], layout=widgets.Layout(margin="0 0 0 auto"))
        self.ndays_slider.children[-1].observe(self._on_ndays_slider_change, names='value')
        self.country_dropdown = widgets.Dropdown(
            options=sorted(set(icDF.location)),
            value = self.country,
            description='Country:',
            disabled=False
        )
        self.country_dropdown.observe(self._on_country_dropdown_change, names='value')

        self.container.children = [
            widgets.HBox(
                [self.country_dropdown, self.ndays_slider],
                layout=widgets.Layout(padding='40px 0 25px 0')
            ),
            self.output_widget
        ]
        
        
    def _on_ndays_slider_change(self, change):
        """
        Called whenever the rolling average window size changes.

        Recompute rolling average and redraw the plot.
        """
        self.ndays = change.new
        self.redraw_whole_plot()

    def _on_country_dropdown_change(self, change):
        """
        Called whenever the country selector changes.

        Pull data for selected country and redraw the plot.
        """
        self.country = change.new
        self.redraw_whole_plot()

    def redraw_whole_plot(self):
        """
        Redraw canvas
        """
        out = self.output_widget
        with out:
            localDF = icDF[icDF['location'] == self.country].dropna(subset=['new_cases'])['new_cases']

            fig, ax = plt.subplots(1)
            out.clear_output(wait=True)
            ax.set_xlim([0, 400])
            ax.set_ylim([-1, 1])
            ax.set_title(f"{self.country} - new_cases change rolling avg. autocorrelation, {self.ndays} days")

            if len(localDF > 10):
                pd.plotting.autocorrelation_plot(localDF.diff().rolling(self.ndays).mean()[self.ndays:], ax=ax)
                
            else:
                ax.text(150, 0, "Insufficient data", fontsize=20)
            plt.show()

app = AutoCorrDisplay()
app.container